In [24]:
from sklearn.datasets import make_blobs
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from sklearn import metrics
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [25]:
data = pd.read_csv("data_cleaned.csv")
data = data.sample(n=10000, random_state=1)
data

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q91,Q92,Q93,Q94,Q95,Q96,Q97,Q98,Q99,DLEAVING
46608,5,5,5,5,5,5,5,5,5,5,...,1,2,1,5,3,5,5,5,5,1
11518,2,4,5,5,2,4,5,4,5,4,...,1,2,1,2,2,5,5,5,5,1
80411,4,3,4,4,2,4,4,5,4,3,...,2,2,5,4,2,5,5,5,5,0
23295,5,4,4,5,4,4,4,4,4,4,...,3,2,2,4,1,4,4,5,5,0
40034,3,2,4,4,4,3,4,3,4,2,...,3,2,1,4,1,5,4,5,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17879,5,4,3,4,4,4,4,2,5,5,...,1,2,1,3,2,5,4,5,5,1
44155,3,4,4,3,2,4,4,3,3,3,...,1,1,1,1,2,4,1,4,4,0
26669,4,4,4,4,4,4,4,4,4,3,...,1,2,1,3,3,4,4,4,4,1
86130,3,4,4,4,4,4,4,5,4,3,...,2,2,4,4,2,5,5,5,5,0


In [26]:
from sklearn.model_selection import train_test_split
y=data['DLEAVING']
X=data.iloc[:, :-1]
X_train, X_test, y_train, y_test= train_test_split(
                                                X, y,
                                                test_size=0.2,
                                                random_state=53)



In [27]:
from sklearn.linear_model import LogisticRegression

reg = LogisticRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)
# For AUC-ROC, you need the probabilities of the positive class
y_score = reg.predict_proba(X_test)[:, 1]
# Calculate precision, recall, f1-score, and support for each class
metrics = precision_recall_fscore_support(y_test, y_pred)

# Calculate macro and weighted averages
macro_avg = precision_recall_fscore_support(y_test, y_pred, average='macro')
weighted_avg = precision_recall_fscore_support(y_test, y_pred, average='weighted')
accuracy = accuracy_score(y_test, y_pred)

# Calculate AUC score, using the scores for the positive class
# If y_score is 2-dimensional (e.g., [n_samples, 2] for binary outcomes), use the second column: y_score[:, 1]
# If y_score is 1-dimensional (e.g., [n_samples]), use it directly
auc_score = roc_auc_score(y_test, y_score[:, 1]) if len(y_score.shape) > 1 else roc_auc_score(y_test, y_score)

# Prepare metrics for DataFrame creation, including AUC
metrics_list = list(metrics) + [[auc_score] * len(metrics[0])]  # Replicate AUC score for each class

# Create DataFrame
df_metrics = pd.DataFrame(metrics_list, 
                          index=['Precision', 'Recall', 'F1-score', 'Support', 'AUC']).T

# Initialize Accuracy column with NaNs or blanks
df_metrics['Accuracy'] = ' '  # This avoids repeating the accuracy for each class

# Adjust additional metrics dictionary to include the AUC and place Accuracy only in the macro average row
additional_rows = [
    {'Class': 'macro avg', 'Precision': macro_avg[0], 'Recall': macro_avg[1], 'F1-score': macro_avg[2], 'Support': ' ', 'AUC': auc_score, 'Accuracy': accuracy},
    {'Class': 'weighted avg', 'Precision': weighted_avg[0], 'Recall': weighted_avg[1], 'F1-score': weighted_avg[2], 'Support': ' ', 'AUC': auc_score, 'Accuracy': ' '}
]

# Convert dictionary to DataFrame and concatenate with the existing one
df_additional = pd.DataFrame(additional_rows)
df_metrics_final = pd.concat([df_metrics, df_additional], ignore_index=True)

# Adjust 'Class' column
df_metrics_final['Class'] = df_metrics_final.index.map(lambda x: f'Class {x}' if x < len(df_metrics) else df_metrics_final.loc[x, 'Class'])

# Rearrange columns to match desired output, including AUC and selectively showing Accuracy
df_metrics_final = df_metrics_final[['Class', 'Precision', 'Recall', 'F1-score', 'AUC', 'Accuracy', 'Support']]


print(df_metrics_final)
df_metrics_final.to_csv("logi10k.csv", index=False)

          Class  Precision    Recall  F1-score       AUC Accuracy Support
0       Class 0   0.834661  0.628186  0.716852  0.873241            667.0
1       Class 1   0.834446  0.937734  0.883080  0.873241           1333.0
2     macro avg   0.834554  0.782960  0.799966  0.873241   0.8345        
3  weighted avg   0.834518  0.834500  0.827643  0.873241                 


C:\Users\diego\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [28]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb



param_grid = {'max_depth':range(1, 5),
                'learning_rate':(0.005, 0.05, 0.5)}
 
grid = GridSearchCV(xgb.XGBClassifier(random_state=0), 
                    param_grid=param_grid, cv=10, verbose=1, scoring='accuracy')
    
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)


# For AUC-ROC, you need the probabilities of the positive class
y_score = grid.predict_proba(X_test)[:, 1]
# Calculate precision, recall, f1-score, and support for each class
metrics = precision_recall_fscore_support(y_test, y_pred)

# Calculate macro and weighted averages
macro_avg = precision_recall_fscore_support(y_test, y_pred, average='macro')
weighted_avg = precision_recall_fscore_support(y_test, y_pred, average='weighted')
accuracy = accuracy_score(y_test, y_pred)

# Calculate AUC score, using the scores for the positive class
# If y_score is 2-dimensional (e.g., [n_samples, 2] for binary outcomes), use the second column: y_score[:, 1]
# If y_score is 1-dimensional (e.g., [n_samples]), use it directly
auc_score = roc_auc_score(y_test, y_score[:, 1]) if len(y_score.shape) > 1 else roc_auc_score(y_test, y_score)

# Prepare metrics for DataFrame creation, including AUC
metrics_list = list(metrics) + [[auc_score] * len(metrics[0])]  # Replicate AUC score for each class

# Create DataFrame
df_metrics = pd.DataFrame(metrics_list, 
                          index=['Precision', 'Recall', 'F1-score', 'Support', 'AUC']).T

# Initialize Accuracy column with NaNs or blanks
df_metrics['Accuracy'] = ' '  # This avoids repeating the accuracy for each class

# Adjust additional metrics dictionary to include the AUC and place Accuracy only in the macro average row
additional_rows = [
    {'Class': 'macro avg', 'Precision': macro_avg[0], 'Recall': macro_avg[1], 'F1-score': macro_avg[2], 'Support': ' ', 'AUC': auc_score, 'Accuracy': accuracy},
    {'Class': 'weighted avg', 'Precision': weighted_avg[0], 'Recall': weighted_avg[1], 'F1-score': weighted_avg[2], 'Support': ' ', 'AUC': auc_score, 'Accuracy': ' '}
]

# Convert dictionary to DataFrame and concatenate with the existing one
df_additional = pd.DataFrame(additional_rows)
df_metrics_final = pd.concat([df_metrics, df_additional], ignore_index=True)

# Adjust 'Class' column
df_metrics_final['Class'] = df_metrics_final.index.map(lambda x: f'Class {x}' if x < len(df_metrics) else df_metrics_final.loc[x, 'Class'])

# Rearrange columns to match desired output, including AUC and selectively showing Accuracy
df_metrics_final = df_metrics_final[['Class', 'Precision', 'Recall', 'F1-score', 'AUC', 'Accuracy', 'Support']]

print(df_metrics_final)
df_metrics_final.to_csv("xgb10k.csv", index=False)

Fitting 10 folds for each of 12 candidates, totalling 120 fits
          Class  Precision    Recall  F1-score       AUC Accuracy Support
0       Class 0   0.829749  0.694153  0.755918  0.879778            667.0
1       Class 1   0.858530  0.928732  0.892252  0.879778           1333.0
2     macro avg   0.844139  0.811443  0.824085  0.879778   0.8505        
3  weighted avg   0.848931  0.850500  0.846785  0.879778                 


In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, classification_report
import pandas as pd
import numpy as np

# Define a simple neural network model for binary classification
model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), activation='relu'),  # Input layer
    Dense(64, activation='relu'),  # Hidden layer
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Make predictions on the testing dataset
y_pred = model.predict(X_test).ravel()  # Flatten the array to 1D if it's not already

# Convert probabilities to binary class labels
pred_nn = np.round(y_pred).astype(int)

# Calculate precision, recall, f1-score, and support for each class
metrics = precision_recall_fscore_support(y_test, pred_nn)

# Calculate macro and weighted averages
macro_avg = precision_recall_fscore_support(y_test, pred_nn, average='macro')
weighted_avg = precision_recall_fscore_support(y_test, pred_nn, average='weighted')
accuracy = accuracy_score(y_test, pred_nn)

# Calculate AUC score directly using y_pred
auc_score = roc_auc_score(y_test, y_pred)

# Create DataFrame
df_metrics = pd.DataFrame(metrics_list, 
                          index=['Precision', 'Recall', 'F1-score', 'Support', 'AUC']).T

# Initialize Accuracy column with NaNs or blanks
df_metrics['Accuracy'] = ' '  # This avoids repeating the accuracy for each class

# Adjust additional metrics dictionary to include the AUC and place Accuracy only in the macro average row
additional_rows = [
    {'Class': 'macro avg', 'Precision': macro_avg[0], 'Recall': macro_avg[1], 'F1-score': macro_avg[2], 'Support': ' ', 'AUC': auc_score, 'Accuracy': accuracy},
    {'Class': 'weighted avg', 'Precision': weighted_avg[0], 'Recall': weighted_avg[1], 'F1-score': weighted_avg[2], 'Support': ' ', 'AUC': auc_score, 'Accuracy': ' '}
]

# Convert dictionary to DataFrame and concatenate with the existing one
df_additional = pd.DataFrame(additional_rows)
df_metrics_final = pd.concat([df_metrics, df_additional], ignore_index=True)

# Adjust 'Class' column
df_metrics_final['Class'] = df_metrics_final.index.map(lambda x: f'Class {x}' if x < len(df_metrics) else df_metrics_final.loc[x, 'Class'])

# Rearrange columns to match desired output, including AUC and selectively showing Accuracy
df_metrics_final = df_metrics_final[['Class', 'Precision', 'Recall', 'F1-score', 'AUC', 'Accuracy', 'Support']]

print(df_metrics_final)
df_metrics_final.to_csv("nn10k.csv", index=False)

Epoch 1/10


C:\Users\diego\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7642 - loss: 0.5159
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 941us/step - accuracy: 0.8400 - loss: 0.4016
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 972us/step - accuracy: 0.8333 - loss: 0.4078
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 925us/step - accuracy: 0.8488 - loss: 0.3928
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 915us/step - accuracy: 0.8478 - loss: 0.3864
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 927us/step - accuracy: 0.8540 - loss: 0.3768
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 963us/step - accuracy: 0.8477 - loss: 0.3905
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 926us/step - accuracy: 0.8456 - loss: 0.3881
Epoch 9/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 904us/step - accuracy: 0.8515 - loss: 0.3756
Epoch 10/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 903us/step - accuracy: 0.8430 - loss: 0.3908
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
          Class  Precision    Recall  F1-score       AUC Accuracy Support
0       Class 0   0

In [30]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
pipeline = Pipeline([
    ('clf', LinearSVC(dual=False))  # 'clf' is the name we give to the LinearSVC step
])



# Define a very simple parameter grid, could be expanded based on need
param_grid = {
    'clf__penalty': ['l1', 'l2'],
    'clf__loss': ['squared_hinge'],
    'clf__tol': [1e-4, 1e-3, 1e-2],
    'clf__C': [0.01, 0.1, 1, 10, 100],
    'clf__max_iter': [1000, 2000, 3000],
    'clf__intercept_scaling': [1, 10, 100]
}


# Initialize GridSearchCV with the SVC estimator and the simple param grid
grid = RandomizedSearchCV(pipeline, param_grid, verbose=3, cv=10)

# Train the model on the training dataset
grid.fit(X_train, y_train)

# Make predictions on the testing dataset using the best model found
y_pred = grid.predict(X_test)

# For AUC-ROC, you need the probabilities of the positive class
y_scores = grid.decision_function(X_test)
# Calculate precision, recall, f1-score, and support for each class
metrics = precision_recall_fscore_support(y_test, y_pred)

# Calculate macro and weighted averages
macro_avg = precision_recall_fscore_support(y_test, y_pred, average='macro')
weighted_avg = precision_recall_fscore_support(y_test, y_pred, average='weighted')
accuracy = accuracy_score(y_test, y_pred)

# Calculate AUC score, using the scores for the positive class
# If y_score is 2-dimensional (e.g., [n_samples, 2] for binary outcomes), use the second column: y_score[:, 1]
# If y_score is 1-dimensional (e.g., [n_samples]), use it directly
auc_score = roc_auc_score(y_test, y_score[:, 1]) if len(y_score.shape) > 1 else roc_auc_score(y_test, y_score)

# Create DataFrame
df_metrics = pd.DataFrame(metrics_list, 
                          index=['Precision', 'Recall', 'F1-score', 'Support', 'AUC']).T

# Initialize Accuracy column with NaNs or blanks
df_metrics['Accuracy'] = ' '  # This avoids repeating the accuracy for each class

# Adjust additional metrics dictionary to include the AUC and place Accuracy only in the macro average row
additional_rows = [
    {'Class': 'macro avg', 'Precision': macro_avg[0], 'Recall': macro_avg[1], 'F1-score': macro_avg[2], 'Support': ' ', 'AUC': auc_score, 'Accuracy': accuracy},
    {'Class': 'weighted avg', 'Precision': weighted_avg[0], 'Recall': weighted_avg[1], 'F1-score': weighted_avg[2], 'Support': ' ', 'AUC': auc_score, 'Accuracy': ' '}
]

# Convert dictionary to DataFrame and concatenate with the existing one
df_additional = pd.DataFrame(additional_rows)
df_metrics_final = pd.concat([df_metrics, df_additional], ignore_index=True)

# Adjust 'Class' column
df_metrics_final['Class'] = df_metrics_final.index.map(lambda x: f'Class {x}' if x < len(df_metrics) else df_metrics_final.loc[x, 'Class'])

# Rearrange columns to match desired output, including AUC and selectively showing Accuracy
df_metrics_final = df_metrics_final[['Class', 'Precision', 'Recall', 'F1-score', 'AUC', 'Accuracy', 'Support']]

print(df_metrics_final)
df_metrics_final.to_csv("svm10k.csv", index=False)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV 1/10] END clf__C=0.1, clf__intercept_scaling=100, clf__loss=squared_hinge, clf__max_iter=2000, clf__penalty=l1, clf__tol=0.001;, score=0.855 total time=   1.2s
[CV 2/10] END clf__C=0.1, clf__intercept_scaling=100, clf__loss=squared_hinge, clf__max_iter=2000, clf__penalty=l1, clf__tol=0.001;, score=0.851 total time=   1.2s
[CV 3/10] END clf__C=0.1, clf__intercept_scaling=100, clf__loss=squared_hinge, clf__max_iter=2000, clf__penalty=l1, clf__tol=0.001;, score=0.845 total time=   1.1s
[CV 4/10] END clf__C=0.1, clf__intercept_scaling=100, clf__loss=squared_hinge, clf__max_iter=2000, clf__penalty=l1, clf__tol=0.001;, score=0.848 total time=   1.3s
[CV 5/10] END clf__C=0.1, clf__intercept_scaling=100, clf__loss=squared_hinge, clf__max_iter=2000, clf__penalty=l1, clf__tol=0.001;, score=0.854 total time=   1.2s
[CV 6/10] END clf__C=0.1, clf__intercept_scaling=100, clf__loss=squared_hinge, clf__max_iter=2000, clf__penalty=l1, c

[CV 1/10] END clf__C=10, clf__intercept_scaling=1, clf__loss=squared_hinge, clf__max_iter=3000, clf__penalty=l1, clf__tol=0.001;, score=0.858 total time=   0.9s
[CV 2/10] END clf__C=10, clf__intercept_scaling=1, clf__loss=squared_hinge, clf__max_iter=3000, clf__penalty=l1, clf__tol=0.001;, score=0.854 total time=   0.9s
[CV 3/10] END clf__C=10, clf__intercept_scaling=1, clf__loss=squared_hinge, clf__max_iter=3000, clf__penalty=l1, clf__tol=0.001;, score=0.846 total time=   1.0s
[CV 4/10] END clf__C=10, clf__intercept_scaling=1, clf__loss=squared_hinge, clf__max_iter=3000, clf__penalty=l1, clf__tol=0.001;, score=0.846 total time=   0.8s
[CV 5/10] END clf__C=10, clf__intercept_scaling=1, clf__loss=squared_hinge, clf__max_iter=3000, clf__penalty=l1, clf__tol=0.001;, score=0.855 total time=   0.9s
[CV 6/10] END clf__C=10, clf__intercept_scaling=1, clf__loss=squared_hinge, clf__max_iter=3000, clf__penalty=l1, clf__tol=0.001;, score=0.850 total time=   0.9s
[CV 7/10] END clf__C=10, clf__inte

In [31]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Adjusting the parameter grid for RandomForestClassifier
param_grid = {
    'n_estimators': [10, 50, 100, 200],  # Number of trees in the forest
    'max_depth': range(1, 5)  # Maximum depth of the tree
}

grid = GridSearchCV(RandomForestClassifier(random_state=0),
                    param_grid=param_grid, cv=10, verbose=1, scoring='accuracy')

# Fit to the training data
grid.fit(X_train, y_train)

# Predict on the test data
y_pred = grid.predict(X_test)

# For AUC-ROC, you need the probabilities of the positive class
y_score = grid.predict_proba(X_test)[:, 1]
# Calculate precision, recall, f1-score, and support for each class
metrics = precision_recall_fscore_support(y_test, y_pred)

# Calculate macro and weighted averages
macro_avg = precision_recall_fscore_support(y_test, y_pred, average='macro')
weighted_avg = precision_recall_fscore_support(y_test, y_pred, average='weighted')
accuracy = accuracy_score(y_test, y_pred)

# Calculate AUC score, using the scores for the positive class
# If y_score is 2-dimensional (e.g., [n_samples, 2] for binary outcomes), use the second column: y_score[:, 1]
# If y_score is 1-dimensional (e.g., [n_samples]), use it directly
auc_score = roc_auc_score(y_test, y_score[:, 1]) if len(y_score.shape) > 1 else roc_auc_score(y_test, y_score)

# Create DataFrame
df_metrics = pd.DataFrame(metrics_list, 
                          index=['Precision', 'Recall', 'F1-score', 'Support', 'AUC']).T

# Initialize Accuracy column with NaNs or blanks
df_metrics['Accuracy'] = ' '  # This avoids repeating the accuracy for each class

# Adjust additional metrics dictionary to include the AUC and place Accuracy only in the macro average row
additional_rows = [
    {'Class': 'macro avg', 'Precision': macro_avg[0], 'Recall': macro_avg[1], 'F1-score': macro_avg[2], 'Support': ' ', 'AUC': auc_score, 'Accuracy': accuracy},
    {'Class': 'weighted avg', 'Precision': weighted_avg[0], 'Recall': weighted_avg[1], 'F1-score': weighted_avg[2], 'Support': ' ', 'AUC': auc_score, 'Accuracy': ' '}
]

# Convert dictionary to DataFrame and concatenate with the existing one
df_additional = pd.DataFrame(additional_rows)
df_metrics_final = pd.concat([df_metrics, df_additional], ignore_index=True)

# Adjust 'Class' column
df_metrics_final['Class'] = df_metrics_final.index.map(lambda x: f'Class {x}' if x < len(df_metrics) else df_metrics_final.loc[x, 'Class'])

# Rearrange columns to match desired output, including AUC and selectively showing Accuracy
df_metrics_final = df_metrics_final[['Class', 'Precision', 'Recall', 'F1-score', 'AUC', 'Accuracy', 'Support']]

print(df_metrics_final)
df_metrics_final.to_csv("rf10k.csv", index=False)

Fitting 10 folds for each of 16 candidates, totalling 160 fits
          Class  Precision    Recall  F1-score       AUC Accuracy Support
0       Class 0   0.829749  0.694153  0.755918  0.879778            667.0
1       Class 1   0.858530  0.928732  0.892252  0.879778           1333.0
2     macro avg   0.807264  0.737983  0.756091  0.859684   0.8045        
3  weighted avg   0.805614  0.804500  0.792276  0.859684                 
